## Algoritmia
### Práctica 8
En esta práctica se plantean dos problemas que se pueden resolver mediante divide y vencerás: 

- Generar todas las posibles maneras de sumar enteros positivos para alcanzar un determinado valor.
- Generar todos los posibles caminos entre dos nodos de un grafo.

Se pide la implementación de las clases y/o funciones que aparecen a continuación.

Las instrucción "pass" que aparecen en el cuerpo de las clases o funciones, se debe sustituir por la implementación adecuada.

Para cada clase o función que se pide se proporciona una o más funciones con algunos tests.

Al llamar a las funciones de test no debería saltar ninguna aserción.

### Obtener un número entero positivo mediante sumas
Dado un número entero positivo, se puede obtener su valor sumando otros enteros positivos. Se quieren obtener todas las posibles maneras. Por ejemplo:

- 4 = 1 + 1 + 1 + 1
- 4 = 1 + 1 + 2
- 4 = 1 + 3
- 4 = 2 + 2

Se va a considerar que el orden en el que aparecen los enteros no importa y por tanto se entiende que dos soluciones con los mismos valores pero en distinto orden son la misma. Por ejemplo, 1 + 1 + 2 es la misma solución que 1 + 2 + 1 y 2 + 1 + 1.

Se puede plantear mediante divide y vencerás. Para obtener el valor n se consideran n-1 subproblemas, donde el primer sumando es i (para i=1 hasta n-1) y las maneras de obtener n-i se obtienen recursivamente.

In [7]:
#Hemos usado lru_cache porque a la hora de pasar los test, para los primeros casos funcionaba
#pero para los casos más complicados se quedaba pillado el notebook, por ello usamos lru_cache
from functools import lru_cache

@lru_cache(10000)
def sumandos(n,inicial=None):
    """
    Dado un número entero positivo n, genera todas las posibles maneras de 
    obtener ese valor mediante sumas de enteros positivos.
    Los valores generados son tuplas de enteros, en orden ascendente, cuya suma
    es el valor objetivo.
    Las tuplas se generan en orden lexicográfico.
    """
    
    lista = []
    if inicial == None:
        inicial = n
    if inicial == 1:
        return tuple()
    if n != 1:
        for i in range(1,n):
            lista_aux = []
            lista_aux.append(i)
            lis = sumandos(n-i,inicial)
            for j in lis:
                if j[-1]<=lista_aux[0]:
                    lista.append(j+tuple(lista_aux))
        if inicial != n:
            lista.append((n,))
    else:
        lista.append((1,))
        
    lista.sort()
    return lista
    

#### Casos de prueba para `sumandos`

In [8]:
def test_sumandos():
    """
    Casos de prueba para la función sumandos.
    """
   
    assert tuple(sumandos(1)) == ()
    
    assert tuple(sumandos(2)) == ((1, 1),)
    
    assert tuple(sumandos(3)) == ((1, 1, 1), (1, 2))
    
    assert tuple(sumandos(4)) == ((1, 1, 1, 1), (1, 1, 2), (1, 3), (2, 2))
    
    assert tuple(sumandos(5)) == ((1, 1, 1, 1, 1), (1, 1, 1, 2), (1, 1, 3), 
                                  (1, 2, 2), (1, 4), (2, 3))
    
    assert tuple(sumandos(6)) == ((1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 2), 
                                  (1, 1, 1, 3), (1, 1, 2, 2), (1, 1, 4), 
                                  (1, 2, 3), (1, 5), (2, 2, 2), (2, 4), (3, 3))

    # número de maneras diferentes de obtener un determinado valor mediante sumas:
    maneras = [0, 0, 1, 2, 4, 6, 10, 14, 21, 29, 41, 55, 76, 100, 134, 175, 230, 
               296, 384, 489, 626, 791, 1001, 1254, 1574, 1957, 2435, 3009, 3717, 
               4564, 5603]
       
    for i in range(1, len(maneras)):
        numero = 0
        previa = ()
        for sums in sumandos(i):
            
            # los valores suman el valor correspondiente:
            assert sum(sums) == i 
            
            # los valores de la tupla están ordenados
            assert all(sums[i] <= sums[i + 1] for i in range(len(sums) - 1))
            
            # las tuplas se generan en orden lexicográfico
            assert previa < sums 
            previa = sums
            
            numero += 1
        
        # el número de maneras es el esperado
        assert numero == maneras[i] 
    
if __name__ == "__main__": 
    test_sumandos()
    print("OK")

OK


### Obtener todos los caminos entre dos nodos de un grafo
Dado un grafo y dos nodos del mismo, se quieren obtener todos los caminos distintos entre esos dos nodos. En estos caminos no se admiten ciclos y por tanto un nodo no puede aparecer más de una vez.

Se puede abordar mediante divide y vencerás. Para cada nodo vecino al que podemos acceder por un arco desde el nodo origen resolvemos el subproblema de generar todos los caminos entre ese vecino y el destino. Al resolver un subproblema hay que tener en cuenta que nodos forman parte del camino acumulado y por tanto no se pueden usar.

Se usarán las implementaciones de grafos desarrolladas en la práctica 2. Dichas implementaciones deben estar en un fichero denominado "`grafos.py`" que  también hay que entregar.


In [9]:
def subgrafo(grafo,origen):
    compis = []
    new = []
    copiaGrafo = GrafoMatriz(grafo.dirigido())
    for j in grafo:
        if j!=origen:
            vecinos = grafo.vecinos(j)
            for i in vecinos:
                if i[0]!=origen:
                    copiaGrafo.inserta(j,i[0],i[1])
    return copiaGrafo

In [10]:
from grafos import GrafoMatriz, GrafoListas
import copy

def caminos(grafo, origen, destino):
    """
    Genera todos los posibles caminos entre el nodo origen y el destino.
    Los caminos serán tuplas de nodos, sin nodos repetidos.
    """
    lista = []
    auxiliar = []
    copiaGrafo = subgrafo(grafo,origen)
    
    if origen == destino:
        return [(destino,)]
    else:
        for vec in grafo.vecinos(origen):
            inicio = tuple(vec[0])
            auxiliar = caminos(copiaGrafo,inicio[0],destino)
            for aux in auxiliar:
                lista.append((origen,)+aux)
        return lista
         

#### Casos de prueba para `caminos`

In [11]:
def test_caminos_no_dirigido(): 
    """
    Casos de prueba para la función caminos con un grafo no dirigido.
    """
    
    arcos = (("c", "d"), ("e", "f"), ('a', 'b'), ('a', 'c'), ('a', 'd'),
             ('b', 'c'), ('c', 'e'), ('d', 'e'), ('d', 'f'))
    
    for claseGrafo in (GrafoMatriz, GrafoListas):
    
        g = claseGrafo(dirigido = False)

        for (origen, destino) in arcos:
            g.inserta(origen, destino)

        assert len(g) == 6
        assert g.num_arcos() == 9
        assert set(caminos(g, "c", "d")) == {('c', 'e', 'f', 'd'), ('c', 'd'), 
            ('c', 'a', 'd'), ('c', 'e', 'd'), ('c', 'b', 'a', 'd')}

        assert set(caminos(g, "b", "c")) == {('b', 'a', 'd', 'c'), 
            ('b', 'a', 'c'), ('b', 'a', 'd', 'f', 'e', 'c'), ('b', 'c'), 
            ('b', 'a', 'd', 'e', 'c')}

        assert set(caminos(g, "a", "f")) == {('a', 'b', 'c', 'e', 'f'), 
            ('a', 'c', 'e', 'd', 'f'), ('a', 'd', 'f'), 
            ('a', 'b', 'c', 'e', 'd', 'f'), ('a', 'd', 'e', 'f'), 
            ('a', 'c', 'e', 'f'), ('a', 'c', 'd', 'f'), 
            ('a', 'd', 'c', 'e', 'f'), ('a', 'b', 'c', 'd', 'f'), 
            ('a', 'c', 'd', 'e', 'f'), ('a', 'b', 'c', 'd', 'e', 'f')}
               
    
if __name__ == "__main__": 
    test_caminos_no_dirigido()
    print("OK")

OK


In [12]:
def test_caminos_dirigido(): 
    """
    Casos de prueba para la función caminos con un grafo dirigido.
    """
    
    arcos = (("c", "d"), ("e", "f"), ('a', 'b'), ('a', 'c'), ('a', 'd'),
             ('b', 'c'), ('c', 'e'), ('d', 'e'), ('d', 'f'))
    
    for claseGrafo in (GrafoMatriz, GrafoListas):
    
        g = claseGrafo(dirigido = True)

        for (origen, destino) in arcos:
            g.inserta(origen, destino)

        assert len(g) == 6
        assert g.num_arcos() == 9
       
        assert set(caminos(g, "c", "d")) == {('c', 'd')}
        
        assert set(caminos(g, "c", "f")) == {('c', 'e', 'f'), ('c', 'd', 'f'), 
            ('c', 'd', 'e', 'f')}
        
        assert set(caminos(g, "a", "f")) == {('a', 'b', 'c', 'e', 'f'), 
            ('a', 'd', 'f'), ('a', 'd', 'e', 'f'), ('a', 'c', 'e', 'f'), 
            ('a', 'c', 'd', 'f'), ('a', 'b', 'c', 'd', 'f'), 
            ('a', 'c', 'd', 'e', 'f'), ('a', 'b', 'c', 'd', 'e', 'f')}

        
if __name__ == "__main__": 
    test_caminos_dirigido()
    print("OK")

OK
